In [ ]:
import plotly.offline as pyo
pyo.init_notebook_mode()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2 
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn import preprocessing
import random
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
!pip install visualkeras

In [ ]:
def dataframe_creation():
    df4 = pd.read_csv('../input/ocular-disease-recognition-odir5k/full_df.csv')
    df4['filename']='../input/ocular-disease-recognition-odir5k/preprocessed_images/'+df4['filename']
    df4['Left-Fundus']='../input/ocular-disease-recognition-odir5k/preprocessed_images/'+df4['Left-Fundus']
    df4['Right-Fundus']='../input/ocular-disease-recognition-odir5k/preprocessed_images/'+df4['Right-Fundus']
    df4['Line'] = df4['Left-Diagnostic Keywords']+' | '+df4['Right-Diagnostic Keywords']
    df4 = df4.drop(['filepath','target'], axis=1)
    return df4
df = dataframe_creation()
print(df.shape)
df.head()

In [ ]:
df['Patient Sex'].unique()

# Visualize Data

In [ ]:
count = 1
f = plt.figure(figsize=(50,20))
for Class in df['labels'].unique():
    seg = df[df['labels']==Class]
    address = seg.sample().iloc[0]['filename']
    img = cv2.imread(address)
    #print(img)
    ax = f.add_subplot(2, 5,count)
    ax = plt.imshow(img)
    ax = plt.title(Class,fontsize= 30)
    count = count + 1
plt.suptitle("Retinal Classification", size = 32)
plt.show()

In [ ]:
w , h= 16,16
final_class = 8

In [ ]:
listImg = os.listdir('../input/ocular-disease-recognition-odir5k/preprocessed_images')
string = '../input/ocular-disease-recognition-odir5k/preprocessed_images/'
list2 = list(map(lambda orig_string: string + orig_string , listImg))
indexify =[]
for i in df.index:
    if df.iloc[i]['Left-Fundus'] in list2 and df.iloc[i]['Right-Fundus'] in list2:
        continue
    else:
        indexify.append(i)

In [ ]:
print(df.shape)
df = df.drop(indexify)
print(df.shape)

In [ ]:
from tqdm import tqdm
left = []
for location in tqdm(df.iloc[:]['Left-Fundus']):
    img = cv2.imread(location,0)
    img = cv2.resize(img, (w,h), interpolation = cv2.INTER_AREA)
    img = img.reshape(w,h,1)
    left.append(img)
right = []
for location in tqdm(df.iloc[:]['Right-Fundus']):
    img = cv2.imread(location,0)
    img = cv2.resize(img, (w,h), interpolation = cv2.INTER_AREA)
    img = img.reshape(w,h,1)
    right.append(img)
X1 = np.array(left)
X2 = np.array(right)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
y = np.array(df.iloc[:][['N','D','G','C','A','H','M','O']])
y = np.array(y)
print('Label :   '+str(y.shape))

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten , Conv1D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D,MaxPooling1D
from tensorflow.keras.utils import plot_model
def model_b():
    inp1 = Input(shape=(X1.shape[1],X1.shape[2],1,))
    inp2 = Input(shape=(X2.shape[1],X2.shape[2],1,))
    new_input = Input(shape=(X1.shape[1],X1.shape[2], 3))
    conv1 = Conv2D(3, kernel_size=3, padding ='same', activation='relu')(inp1)
    i1 = tf.keras.applications.ResNet50(include_top=False,weights="imagenet",input_tensor=new_input,
                        input_shape=None,
                         pooling='avg',
                    )(conv1)
    conv2= Conv2D(3, kernel_size=3, padding ='same', activation='relu')(inp2)
    i2 = tf.keras.applications.ResNet50V2(include_top=False,weights="imagenet",input_tensor=new_input,
                        input_shape=None,
                         pooling='avg',
                    
                    )(conv2)
    merge = concatenate([i1,i2])
    class1 = Dense(1024, activation='relu')(merge)
    class1 = Dense(512, activation='relu')(class1)
    class1 = Dense(256, activation='relu')(class1)
    class1 = Dense(128, activation='relu')(class1)
    class1 = Dense(64, activation='relu')(class1)
    output = Dense(final_class, activation='sigmoid')(class1)
    model = Model(inputs=[inp1,inp2], outputs=output)
    return model
md = model_b()

In [ ]:
plot_model(md, to_file='Hybrid_neural_network.png')

# Train Test Splitting

In [ ]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, X_train2, X_test2, y_train, y_test = train_test_split(X1,X2, y, test_size=0.20, random_state=42)

In [ ]:
X_train1, X_valid1, X_train2, X_valid2, y_train, y_valid = train_test_split(X_train1,X_train2, y_train, test_size=0.20, random_state=42)

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
def LR_verify():
        model = model_b()
        METRICS = [
                'accuracy'
        ]  
        model.compile(
                optimizer='Adam',
                loss='binary_crossentropy',
                metrics=METRICS
            )
        history = model.fit([X1,X2], y, epochs=100,verbose=1,shuffle = True, validation_data=([X_valid1,X_valid2],y_valid))
        return model,history
model,history= LR_verify()

In [ ]:
from tensorflow.keras.models import model_from_json
model_json = model.to_json()
name = 'ODIR_ResNet'
with open(name+".json", "w") as json_file:
    json_file.write(model_json)
model.save_weights(name+".h5")
import pandas as pd
hist_df = pd.DataFrame(history.history) 
hist_df.to_csv(name+'.csv',index = False)
hist_df.tail()

In [ ]:
json_file = open('./'+name+'.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("./"+name+".h5")
print("Loaded model from disk")

# Results

## Train

In [ ]:
METRICS = [
                'accuracy',
                tf.keras.metrics.FalseNegatives(),
                tf.keras.metrics.FalsePositives(),
                tf.keras.metrics.Precision(),
                tf.keras.metrics.Recall(),
                tf.keras.metrics.TrueNegatives(),
                tf.keras.metrics.TruePositives()
        ]  
loaded_model.compile(
                optimizer='Adam',
                loss='binary_crossentropy',
                metrics=METRICS
            )
score = loaded_model.evaluate([X1,X2], y, verbose=0)

In [ ]:
for i in range(len(score)):
    print(loaded_model.metrics_names[i]+" : "+str(score[i]))

## Validation

In [ ]:
METRICS = [
                'accuracy',
                tf.keras.metrics.FalseNegatives(),
                tf.keras.metrics.FalsePositives(),
                tf.keras.metrics.Precision(),
                tf.keras.metrics.Recall(),
                tf.keras.metrics.TrueNegatives(),
                tf.keras.metrics.TruePositives()
        ]  
loaded_model.compile(
                optimizer='Adam',
                loss='binary_crossentropy',
                metrics=METRICS
            )
score = loaded_model.evaluate([X_valid1,X_valid2], y_valid, verbose=0)

In [ ]:
for i in range(len(score)):
    print(loaded_model.metrics_names[i]+" : "+str(score[i]))

## Test

In [ ]:
METRICS = [
                'accuracy',
                tf.keras.metrics.FalseNegatives(),
                tf.keras.metrics.FalsePositives(),
                tf.keras.metrics.Precision(),
                tf.keras.metrics.Recall(),
                tf.keras.metrics.TrueNegatives(),
                tf.keras.metrics.TruePositives()
        ]  
loaded_model.compile(
                optimizer='Adam',
                loss='binary_crossentropy',
                metrics=METRICS
            )
score = loaded_model.evaluate([X_test1,X_test2], y_test, verbose=0)

In [ ]:
for i in range(len(score)):
    print(loaded_model.metrics_names[i]+" : "+str(score[i]))

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import pandas as pd
yhat = loaded_model.predict([X_test1,X_test2])
yhat = yhat.round()
report = classification_report(y_test, yhat,target_names=['N','D','G','C','A','H','M','O'],output_dict=True)
df = pd.DataFrame(report).transpose()
df

In [ ]:
df.to_csv('Class_Wise_Accuracy')